In [1]:

# %%
from attribution_utils import calculate_feature_attribution
from collections import defaultdict
from torch.nn.functional import log_softmax
from gemma_utils import get_all_string_min_l0_resid_gemma
from transformer_lens.hook_points import HookPoint
from transformer_lens import utils
from functools import partial
import tqdm
from sae_lens import HookedSAETransformer, SAE, SAEConfig
from gemma_utils import get_gemma_2_config, gemma_2_sae_loader
import numpy as np
import torch
import tqdm
import pandas as pd

import torch.nn.functional as F
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:


model = HookedSAETransformer.from_pretrained("google/gemma-2-2b-it")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model google/gemma-2-2b-it into HookedTransformer


In [3]:

generation_dict = torch.load("gemma2_generation_dict.pt")

In [19]:

def logits_to_entropy(logits):
    log_probs = log_softmax(logits, dim = -1)
    probs = log_probs.exp()
    entropy = -(log_probs*probs).sum(dim = -1)
    return entropy


def logits_to_varentropy(logits):
    log_probs = log_softmax(logits, dim = -1)
    probs = log_probs.exp()
    entropy = -(log_probs*probs).sum(dim = -1)
    elem = (probs*(-log_probs)**2).sum(dim = -1)
    return elem - entropy



In [71]:
model.to_tokens(" ")

tensor([[     2, 235290]], device='cuda:0')

In [93]:

all_entropy_hyphen = []
all_entropy_breaks = []
for key in list(generation_dict.keys()):
    val = generation_dict[key]
    for toks in val:
        with torch.no_grad():
            pos_hyphen = torch.where(toks[0] == 235290)[0]+1
            pos_break = torch.where(toks[0] == 108)[0][3:]
            if len(pos_hyphen) != 10:
                continue
            logits = model(toks)
            entropy = logits_to_entropy(logits)
            all_entropy_hyphen.append(entropy[:,pos_hyphen])
            all_entropy_breaks.append(entropy[:,pos_break])



max_size = max(tensor.size(1) for tensor in all_entropy_hyphen)
padded_tensors = []
for tensor in all_entropy_hyphen:
    pad_amount = max_size - tensor.size(1)
    padded_tensor = F.pad(tensor, (pad_amount, 0), "constant", 0)  # Left padding
    padded_tensors.append(padded_tensor)
stacked_entropy_hyphen = torch.cat(padded_tensors,dim = 0)


max_size = max(tensor.size(1) for tensor in all_entropy_breaks)
padded_tensors = []
for tensor in all_entropy_breaks:
    pad_amount = max_size - tensor.size(1)
    padded_tensor = F.pad(tensor, (pad_amount, 0), "constant", 0)  # Left padding
    padded_tensors.append(padded_tensor)
stacked_entropy_breaks = torch.cat(padded_tensors,dim = 0)





In [94]:

px.imshow(stacked_entropy_breaks.cpu().numpy(), aspect = 'auto')